In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

/home/andres/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")
df.head()

,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0


In [3]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp"
)
train_data.head()

target
item_id timestamp                  
H1      1750-01-01 00:00:00   605.0
        1750-01-01 01:00:00   586.0
        1750-01-01 02:00:00   586.0
        1750-01-01 03:00:00   559.0
        1750-01-01 04:00:00   511.0

In [4]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MASE",
)

predictor.fit(
    train_data,
    presets="fast_training",
    time_limit=600,
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: fast_training
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'MASE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'fast_training',
 'num_val_windows': 1,
 'prediction_length': 48,
 'random_seed': None,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}
Provided training data set with 148060 rows, 200 items (item = single time series). Average time series length is 740.3. Data frequency is 'H'.
AutoGluon will save models to autogluon-m4-hourly/
AutoGluon will gauge predictive performance using evaluation metric: 'MASE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'target'

Starting training. Start time is 2023-11-10 05:47:18
Models that will be tr

In [5]:
predictions = predictor.predict(train_data)
predictions.head()

Global seed set to 123
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/andres/miniforge3/lib/python3.10/site-packages/networkx/classes/digraph.py:927 in          │
│ predecessors                                                                                     │
│                                                                                                  │
│    924 │   │   successors                                                                        │
│    925 │   │   """                                                                               │
│    926 │   │   try:                                                                              │
│ ❱  927 │   │   │   return iter(self._pred[n])                                                    │
│    928 │   │   except KeyError as err:                                                           │
│    929 │   │   │   raise NetworkXError(f"The node {n} is not in the digraph.") from err          │
│    930                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'DeepAR'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictions = predictor.predict(train_data)                                                  │
│   2 predictions.head()                                                                           │
│   3                                                                                              │
│                                                                                                  │
│ /home/andres/miniforge3/lib/python3.10/site-packages/autogluon/timeseries/predictor.py:630 in    │
│ predict                                                                                          │
│                                                                                                  │
│   627 │   │   # Don't use data.item_ids in case data is not a TimeSeriesDataFrame                │
│   628 │   │   original_item_id_order = data.reset_index()[ITEMID].unique()                       │
│   629 │   │   data = self._check_and_prepare_data_frame(data)                                    │
│ ❱ 630 │   │   predictions = self._learner.predict(data, known_covariates=known_covariates, mod   │
│   631 │   │   return predictions.reindex(original_item_id_order, level=ITEMID)                   │
│   632 │                                                                                          │
│   633 │   def evaluate(self, data: Union[TimeSeriesDataFrame, pd.DataFrame], **kwargs):          │
│                                                                                                  │
│ /home/andres/miniforge3/lib/python3.10/site-packages/autogluon/timeseries/learner.py:198 in      │
│ predict                                                                                          │
│                                                                                                  │
│   195 │   │   data = self.feature_generator.transform(data)                                      │
│   196 │   │   known_covariates = self.feature_generator.transform_future_known_covariates(know   │
│   197 │   │   known_covariates = self._align_covariates_with_forecast_index(known_covariates=k   │
│ ❱ 198 │   │   return self.load_trainer().predict(                                                │
│   199 │   │   │   data=data, known_covariates=known_covariates, model=model, use_cache=use_cac   │
│   200 │   │   )                                                       

In [6]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

print(test_data)

plt.figure(figsize=(20, 3))

item_id = "H1"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"][-48:]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend();

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>
MultiIndex: 157660 entries, ('H1', Timestamp('1750-01-01 00:00:00')) to ('H200', Timestamp('1750-02-11 23:00:00'))
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   target  157660 non-null  float64
dtypes: float64(1)
memory usage: 1.9+ MB
None


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9                                                                                             │
│   10 item_id = "H1"                                                                              │
│   11 y_past = train_data.loc[item_id]["target"]                                                  │
│ ❱ 12 y_pred = predictions.loc[item_id]                                                           │
│   13 y_test = test_data.loc[item_id]["target"][-48:]                                             │
│   14                                                                                             │
│   15 plt.plot(y_past[-200:], label="Past time series values")                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictions' is not defined

<Figure size 2000x300 with 0 Axes>

In [ ]:
# The test score is computed using the last
# prediction_length=48 timesteps of each time series in test_data
predictor.leaderboard(test_data, silent=True)